### **Viewpoint 1: Optimistic Perspective — Generative AI is a Disruptive Productivity Booster**

1. **Generative AI will significantly boost global productivity**  
   Supporters of this view argue that generative AI tools like ChatGPT, Copilot, and other large language models will streamline workflows and automate cognitive labor, particularly in sectors like law, software development, education, finance, and media. The result? Workers can focus on higher-level strategic or creative tasks instead of spending time on repetitive processes. Over time, this productivity lift could rival historical tech shifts like the PC or the internet.

2. **Job losses may happen — but new ones will also emerge**  
   Yes, some jobs may be displaced — especially roles that involve routine data entry, transcription, or customer service. But, optimists believe this won’t be a net negative. Just like during past technological revolutions, they expect that generative AI will *create* new job categories too (think AI ethics consultants, prompt engineers, or human-AI collaboration specialists).

3. **Generative AI can lead to economic growth and wage increases**  
   The broader belief here is that if AI increases output across industries, it will also increase company profits and GDP. Higher efficiency and productivity typically lead to greater economic output, and in the long term, this could raise wages — especially for workers who know how to leverage AI effectively.

4. **Current challenges are transitional**  
   This side acknowledges issues like hallucinations, bias, or the need for human oversight — but views these as solvable engineering problems. As models get better and training data improves, many of these concerns will fade with time and iteration.

---

### **Viewpoint 2: Cautious Perspective — Generative AI Has Risks That May Outweigh Benefits**

1. **Generative AI’s actual impact on productivity is uncertain**  
   Critics argue that while AI has shown impressive *potential*, there's limited hard evidence that it meaningfully improves productivity at scale. Much of the productivity gain is theoretical or based on small, controlled experiments — not long-term, real-world deployment. Some fear AI will overpromise and underdeliver, like other hyped technologies before it.

2. **The technology could exacerbate inequality and job loss**  
   This viewpoint emphasizes that low-to-mid skill jobs are the most vulnerable to automation, and the people who benefit most from generative AI may already be in high-paying roles. Without government intervention or retraining programs, this could worsen income inequality and displace millions of workers — especially in developing countries.

3. **Overreliance on AI could degrade human judgment and creativity**  
   Critics fear that widespread AI use could make professionals too reliant on machine-generated outputs. If people stop thinking critically or lose touch with foundational skills (e.g., writing, coding, or analysis), the long-term impact on creativity and decision-making could be negative.

4. **AI’s risks (misinformation, hallucinations, biases) are not trivial**  
   This group stresses that hallucinated facts, biased outputs, and manipulated content (deepfakes, AI-generated propaganda) are not minor bugs — they’re core risks that may undermine trust in institutions, education, and public discourse. The fear is that we’ll scale flawed models faster than we can govern them.

5. **Lack of global regulation is dangerous**  
   Without coordinated global oversight, companies might race ahead to release AI models for competitive advantage, even when they aren’t fully safe. This could mirror the dangers we’ve seen in unregulated financial products, social media algorithms, or carbon emissions — with huge downstream consequences.

---

### **My Take: A Realistic Middle Ground**

I believe both viewpoints have valid concerns. Generative AI clearly holds immense promise — its ability to assist, automate, and enhance human work is real. But that promise can’t blind us to its potential downsides.

- We need to measure impact carefully, not just assume AI will make everything better.
- Regulation and human oversight are **non-negotiable** to ensure responsible deployment.
- Education systems must evolve to teach “AI fluency” — knowing when to trust it, how to collaborate with it, and how to spot its limits.
- And most importantly, AI should be *augmented intelligence*, not a replacement for human reasoning or empathy.

### **Course Completion Certificate**

https://learn.deeplearning.ai/accomplishments/a77813b0-05ec-4cfb-a7d8-1abbd4b191ca?usp=sharing

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import time

# === 1. Load and preprocess Buffett text data ===
with open("Dummy_Text.txt", "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s): return [stoi[c] for c in s]
def decode(t): return ''.join([itos[i] for i in t])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

# === 2. Data loader ===
block_size = 128
batch_size = 64

def get_batch(split):
    source = train_data if split == 'train' else val_data
    ix = torch.randint(len(source) - block_size, (batch_size,))
    x = torch.stack([source[i:i+block_size] for i in ix])
    y = torch.stack([source[i+1:i+block_size+1] for i in ix])
    return x, y

# === 3. Transformer Language Model ===
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size, embedding_dim, block_size):
        super().__init__()
        self.key = nn.Linear(embedding_dim, head_size, bias=False)
        self.query = nn.Linear(embedding_dim, head_size, bias=False)
        self.value = nn.Linear(embedding_dim, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        att = q @ k.transpose(-2, -1) / math.sqrt(k.size(-1))
        att = att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.dropout(att)
        v = self.value(x)
        return att @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, embedding_dim, block_size):
        super().__init__()
        self.heads = nn.ModuleList([
            SelfAttentionHead(head_size, embedding_dim, block_size) for _ in range(num_heads)
        ])
        self.proj = nn.Linear(num_heads * head_size, embedding_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        return self.dropout(self.proj(torch.cat([h(x) for h in self.heads], dim=-1)))

class FeedForward(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_dim, 4 * embedding_dim),
            nn.ReLU(),
            nn.Linear(4 * embedding_dim, embedding_dim),
            nn.Dropout(0.1)
        )

    def forward(self, x): return self.net(x)

class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, block_size):
        super().__init__()
        head_size = embedding_dim // num_heads
        self.sa = MultiHeadAttention(num_heads, head_size, embedding_dim, block_size)
        self.ffwd = FeedForward(embedding_dim)
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ln2 = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class TransformerLM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, block_size, num_heads, num_layers):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Embedding(block_size, embedding_dim)
        self.blocks = nn.Sequential(*[
            TransformerBlock(embedding_dim, num_heads, block_size) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx)
        pos_emb = self.pos_embedding(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        return self.head(x)

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx

# === 4. Training ===
device = "cuda" if torch.cuda.is_available() else "cpu"
model = TransformerLM(vocab_size, 256, block_size, num_heads=4, num_layers=4).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

def estimate_loss():
    model.eval()
    losses = {}
    for split in ['train', 'val']:
        total = 0
        for _ in range(5):
            X, Y = get_batch(split)
            X, Y = X.to(device), Y.to(device)
            with torch.no_grad():
                logits = model(X)
                loss = F.cross_entropy(logits.view(-1, vocab_size), Y.view(-1))
            total += loss.item()
        losses[split] = total / 5
    model.train()
    return losses

# === Training loop ===
max_iters = 2500
eval_interval = 500
print("Starting training...")
start_time = time.time()

for step in range(max_iters):
    xb, yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)

    logits = model(xb)
    loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % eval_interval == 0 or step == max_iters - 1:
        elapsed = time.time() - start_time
        losses = estimate_loss()
        print(f"Step {step:4d} | Train Loss: {losses['train']:.4f} | Val Loss: {losses['val']:.4f} | Elapsed: {elapsed:.1f}s")

# === 5. Perplexity ===
val_loss = estimate_loss()['val']
perplexity = math.exp(val_loss)
print(f"\n Final Perplexity: {perplexity:.2f}")

# === 6. Generate Buffett-style text ===
context = torch.tensor([[stoi['B']]], dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=1000)[0].tolist()
generated_text = decode(generated)

print("\n === GENERATED TEXT ===\n")
print(generated_text)

# Save the generated text to a file
with open("generated_buffett_style.txt", "w", encoding="utf-8") as f:
    f.write(decode(generated))

Starting training...
Step    0 | Train Loss: 3.8465 | Val Loss: 3.8325 | Elapsed: 0.4s
Step  500 | Train Loss: 1.5288 | Val Loss: 1.6361 | Elapsed: 221.8s
Step 1000 | Train Loss: 1.2268 | Val Loss: 1.4082 | Elapsed: 438.8s
Step 1500 | Train Loss: 1.0511 | Val Loss: 1.3616 | Elapsed: 653.0s
Step 2000 | Train Loss: 0.9190 | Val Loss: 1.3889 | Elapsed: 878.1s
Step 2499 | Train Loss: 0.8159 | Val Loss: 1.4184 | Elapsed: 1099.1s

 Final Perplexity: 4.00

 === GENERATED TEXT ===

Both 
Berkadia. Who need to a human portfolio inep and capital look both prospects were do that indigible dealing about 
itself. Or IBM's shareholder. We expected to earnings derivatives precise and even untitors that day. 


24 


Term a few of obtable Culture its annual advertisition, I continue to wance out international in 1985, Ajit has grock, and 
other of the books are unfalled us one business to hold a faction resural gains for the yard? 


A lows greater CEO customers at least 18 companies and 98%. In earli

## Evaluation of Transformer Language Model on Buffett's Letters

### Performance Evaluation: Model Perplexity (5 points)

After training the Transformer language model for 2500 steps on Warren Buffett’s annual letters, the model achieved a **final validation perplexity of 4.00**. 

Perplexity is a standard metric in language modeling that measures how well a probability model predicts a sample. A lower perplexity score indicates better performance. Given the limited dataset and character-level modeling, a perplexity below 4 suggests that the model learned meaningful structure in the text and can predict upcoming characters with decent confidence.

This level of perplexity means the model has successfully captured many of the underlying patterns in Buffett’s writing, including domain-specific terminology and stylistic conventions.

---

### Most Impressive Generated Text & Design Insights (5 points)

Below is a sample of the generated text after training:

Both 
Berkadia. Who need to a human portfolio inep and capital look both prospects were do that indigible dealing about 
itself. Or IBM's shareholder. We expected to earnings derivatives precise and even untitors that day. 


24 


Term a few of obtable Culture its annual advertisition, I continue to wance out international in 1985, Ajit has grock, and 
other of the books are unfalled us one business to hold a faction resural gains for the yard? 


A lows greater CEO customers at least 18 companies and 98%. In earlier in booked variety gains and worker 
only out. Here is not the future otilities, ownership. 


And now, though I have an ink only out the respect outla
me. Wills they large my best making an acquisition. The was "guareas our componsed to have iastitutional 
requivate require with Berkshire's book vastasticals 
of a low own ship of $1.4 billion in compounding (recording retroactive reinsurance component to thouse of it good timm, teamble her), only 
a nuteworting insurance b

While some of the output contains gibberish due to the character-level nature and limited data, the model still demonstrates remarkable stylistic alignment:
- Repetitive financial phrases: “carrying value,” “insurance,” “manager,” “losses”
- Structured sentence framing similar to Buffett’s report tone
- Domain-specific themes around business operations and financial results

---

### High-Impact Design Choices

1. **Character-Level Modeling:**  
   Despite limited data, using a character-level tokenizer allowed the model to learn granular linguistic and stylistic nuances — which is critical in mimicking a specific author’s writing.

2. **Causal Masking for Autoregressive Generation:**  
   The use of attention masks ensured tokens are predicted based only on past context, mimicking true language modeling and maintaining coherence.

3. **Multi-Head Attention with Positional Embeddings:**  
   Enabled the model to capture long-range dependencies and order-sensitive phrasing, which is essential in financial narrative generation.

4. **Layer Normalization and Dropout:**  
   Helped regularize the model, improving generalization and stability during training, especially given the relatively small dataset.

5. **Training Monitoring with Perplexity:**  
   Regular evaluation using perplexity guided tuning and early-stopping considerations, preventing overfitting.

---

In summary, even with simple architecture and modest training, the model effectively mirrors Buffett’s thematic and stylistic essence. With more data and longer training, it could produce even more fluent and business-aware narratives.